In [91]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import csv
from datetime import datetime

* Mood vector Design 1: for each day, if negative_count > positive_count or negative_count > neutral_count or neative_count > mix_count, assign negative to that day; if positive_count > negative_count or positive_count > neutral_count or positive_count > mix_count, assign positive to that day; if mix_count >= positive_count or mix_count >= negative_count or negative_count == positive_count, assign mix to that day; if neutral_count >= positive_count or neutral_count >= negative_count or neutral_count > mix_count, assign neutral to that day.

In [153]:
#path = '/Users/lucia/phd_work/cognitive_distortion/'
path = '/home/lucia/phd_work/mypersonality_data/cognitive_distortion/'
AllP = pd.read_csv(path + '/data/important_data/twoM_newLabels2.csv')
ids = pd.read_csv(path + '/data/important_data/FinalSampleUsers.csv')
#remove underage and ppl with less than 80% of posts retained
AllP  = AllP [AllP ['userid'].isin(ids['userid'])]
print(AllP.shape)
lyrics = pd.read_csv(path + '/data/important_data/lyrics/QuotesDetected_all.csv')
lyrics = lyrics[['text','label']]
time = pd.merge(lyrics, AllP, on='text', how = 'right')
print(time.shape)
time['label'] = time['label'].replace(np.nan, 'NotQuote', regex=True)
time = time[['text', 'userid', 'time', 'time_diff', 'id', 'label']]
time['label'].replace(['NotQuote','quote','suspect'],[2,1,1],inplace=True)
time.to_csv(path + '/data/important_data/lyrics/time.csv')

(3838, 14)
(3838, 15)


In [155]:
len(time[time['label'] == 2])

3577

In [156]:
#select date
time['time'] = time['time'].apply(lambda x: x.split()[0])
time[0:5]

,text,userid,time,time_diff,id,label
0,"I've got an ace up my sleeve, and a jack in my...",8ff6192bbbb717c36097b33e244d91de,04/02/2011,53,3055,2
1,I've got cold feet.,f49956142aab2ad74decfd49a103c2f1,12/01/2011,38,904,2
2,"I've got hoes I've got hoes, in different area...",009d96a823b6f6c085c092fb177491f6,16/03/2011,57,957,2
3,I've got the electric tea kettle. I need tea a...,f49956142aab2ad74decfd49a103c2f1,23/12/2010,58,906,2
4,I've gotten up from my desk three times to get...,0e3fdb89c7706b13891170ab2a9017c7,18/03/2011,5,1128,2


In [157]:
#convert to time series
# time['time'] = time['time'].apply(lambda x: parser.parse(x))
# time['time'] = time['time'].apply(lambda x: datetime.strptime(x, '%d %b %Y'))
time['time'] = time['time'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))
time['time'][1:10]

1   2011-01-12
2   2011-03-16
3   2010-12-23
4   2011-03-18
5   2011-01-07
6   2011-03-10
7   2011-05-11
8   2011-02-07
9   2011-01-11
Name: time, dtype: datetime64[ns]

In [158]:
#sort values according to userid, time and valence
time = time.sort_values(by=['userid','time_diff','label'],  ascending=False)
#time.to_csv(path + '/data/important_data/time.csv')

In [159]:
def getTransitions(LyricObject):
    emptyTran = 0
    lyricsTran = 0
    lyricsEmpty = 0
    Original = 0
    LyricsOriginal = 0 
    OriginalEmpty = 0 
    preValence = 0
    for valence in LyricObject:
    #these are self transition states
        if valence == 0 and preValence == 0:
            emptyTran = emptyTran + 1
            
        elif valence == 1 and preValence == 1:
            lyricsTran = lyricsTran + 1
            
        elif (valence == 0 and preValence) == 1 or (valence == 1 and preValence == 0):
            lyricsEmpty = lyricsEmpty + 1
            
        elif valence == 2 and preValence == 2:
            Original = Original + 1
            
        elif (valence == 1 and preValence == 2) or  (valence == 2 and preValence == 1):
            LyricsOriginal = LyricsOriginal + 1
            
        elif (valence == 2 and preValence == 0) or  (valence == 0 and preValence == 2):
            OriginalEmpty = OriginalEmpty + 1 
            
        preValence = valence
  
    return [emptyTran, lyricsTran, lyricsEmpty, Original, LyricsOriginal, OriginalEmpty, preValence]


all the users should start with 60 days, if not, that's because their posts contain foreign lanaguge and they were cleaned from the data set, in this case, the valence vector starts with the first post


In [160]:
def getLyricsVector(df):
    valenceVec = {}
    valences =[]
    preUser = None
    preDay = None
    preValence = []

    for valence, day, user in zip(df['label'], df['time_diff'], df['userid']):
        if preUser is None:
            if day < 60: #if first day is not 60 
                addDays = 60 - day
                for num in range(0, addDays):
                    valences.append(0)
                valenceVec[user] = valences
            else:
                valences = [valence]
                valenceVec[user] = valences

        elif user == preUser and preDay == day+1:
            valences.append(valence)
            
        elif user == preUser and preDay != day+1:
            valences.append(valence)
            addDays = preDay - (day+1)
            for num in range(0, addDays):
                valences.append(0)
                
        elif user != preUser:
            if day < 60: #if first day is not 60 
                addDays = 60 - day
               # print(user,addDays)
                valences = [valence]
                for num in range(0, addDays):
                    valences.insert(0, 0)
                valenceVec[user] = valences
                #print(valences)
            else:
                valences = [valence]
                valenceVec[user] = valences
                #print(valences)
                
            if preDay > 1:
                addDays = preDay
                for num in range(0, addDays):
                    preValence.append(0)
                    
        preUser = user
        preDay = day
        preValence = valences
#     valenceVec[preUser] = valences
    return valenceVec

valenceVec = getLyricsVector(time)

In [161]:
i = 0
for item in valenceVec:
    print(item, valenceVec[item])
    i = i+1
print(i)


f8708568eff1535c85247b28dc126f9b [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
f49956142aab2ad74decfd49a103c2f1 [0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 2, 0, 0, 0, 2, 2, 2, 0, 2, 2, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0]
ec3dc451b8a811f05ace158a0d76e32e [0, 0, 0, 0, 2, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 2, 2, 1, 2, 0, 0, 2, 2, 2

In [162]:
def countLyrics(vector,val):
    count = 0
    mylist = []
    for userid in valenceVec:
        mylist.append(valenceVec[userid].count(val))
    return mylist


empty = countLyrics(valenceVec,0)
lyrics = countLyrics(valenceVec,1)


In [122]:
def getUserTransitions(valencVec):
    result = {}
    for item in valencVec:
        result[item] = getTransitions(valencVec[item])
#         print(result)
    return result
        
TransitionStates = getUserTransitions(valenceVec)   


create table for statistics

In [123]:
df = pd.DataFrame(np.array(empty).reshape(67,1), columns=['emptyDays'])
df['lyrics'] = lyrics
df['userid'] = valenceVec.keys()

In [124]:
df.describe()

,emptyDays,lyrics
count,67.000000,67.000000
mean,56.358209,0.597015
std,5.719985,1.101785
min,18.000000,0.000000
25%,56.000000,0.000000
50%,58.000000,0.000000
75%,59.000000,1.000000
max,60.000000,5.000000


In [63]:
# #pickle
# import pickle
# with open(path + 'newScripts/moodVector/moodVectorsData/MoodVecDes3Empty.pickle', 'wb') as handle:
#     pickle.dump(valenceVec, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [69]:
#write result as csv
#header = ['userid','vector']
with open(path + 'newScripts/moodVector/moodVectorsData/PostsLyricsTransitions(empty).csv','w') as csv_file:
    writer = csv.writer(csv_file)
   # writer.writerow(i for i in header)
    writer.writerow(TransitionStates.keys())
    for row in zip(*TransitionStates.values()):
        writer.writerow(list(row))

In [79]:
#read csv file 
file = pd.read_csv(path + 'newScripts/moodVector/moodVectorsData/PostsLyricsTransitions(empty).csv')
file = file.transpose()
file.columns = ['emptyTrans','lyricsTran', 'lyricsEmpty']
file.shape

(67, 3)

In [80]:
#add one column as row sum
file['allTrans'] = file.sum(axis=1) 
file[1:10]

,emptyTrans,lyricsTran,lyricsEmpty,allTrans
fbd681e79ebca2cd9509218403f23ec6,59,0,2,61
f8708568eff1535c85247b28dc126f9b,61,0,0,61
f49956142aab2ad74decfd49a103c2f1,61,0,0,61
f358b75c71d85c91ec0452c6114e3b15,61,0,0,61
eaeabcfe25866f0455d349bae2450d88,61,0,0,61
e076953cd3fcf8927b8f21574c0890f3,61,0,0,61
dcbe08425f5edef5e5c1053f21e83640,59,0,2,61
db7f1b0130b2138e13b7bbca9cc63823,59,0,2,61
d5380466e18422d7b2c2d54d57cbbde2,59,0,2,61


compute transition states probability,  we compute the pobability by dividing the transition with number of all posts

In [82]:
Tranprob = file.apply(lambda x: x/file.iloc[:,-1])
#Tranprob = file.apply(lambda x: x/1)
Tranprob['userid'] = Tranprob.index
Tranprob[1:20]

,emptyTrans,lyricsTran,lyricsEmpty,allTrans,userid
fbd681e79ebca2cd9509218403f23ec6,0.967213,0.0,0.032787,1.0,fbd681e79ebca2cd9509218403f23ec6
f8708568eff1535c85247b28dc126f9b,1.000000,0.0,0.000000,1.0,f8708568eff1535c85247b28dc126f9b
f49956142aab2ad74decfd49a103c2f1,1.000000,0.0,0.000000,1.0,f49956142aab2ad74decfd49a103c2f1
f358b75c71d85c91ec0452c6114e3b15,1.000000,0.0,0.000000,1.0,f358b75c71d85c91ec0452c6114e3b15
eaeabcfe25866f0455d349bae2450d88,1.000000,0.0,0.000000,1.0,eaeabcfe25866f0455d349bae2450d88
e076953cd3fcf8927b8f21574c0890f3,1.000000,0.0,0.000000,1.0,e076953cd3fcf8927b8f21574c0890f3
dcbe08425f5edef5e5c1053f21e83640,0.967213,0.0,0.032787,1.0,dcbe08425f5edef5e5c1053f21e83640
db7f1b0130b2138e13b7bbca9cc63823,0.967213,0.0,0.032787,1.0,db7f1b0130b2138e13b7bbca9cc63823
d5380466e18422d7b2c2d54d57cbbde2,0.967213,0.0,0.032787,1.0,d5380466e18422d7b2c2d54d57cbbde2
cfb021cfbff46ffdb75c7dcafe7c2087,0.906250,0.0,0.093750,1.0,cfb021cfbff46ffdb75c7dcafe7c2087


In [68]:
#Tranprob.to_csv(path + 'newScripts/moodVector/moodVectorsData/PostsValenceTransitions(Empty).csv')

correlation with all scales

In [83]:
allData = pd.read_csv( path + '/data/important_data/user_scale_post_time2.csv')
Var = allData[['userid','ope','con','ext','agr','neu','swl','CESD_sum']]
Var.shape

(301, 8)

In [86]:
compare = pd.merge(Tranprob, Var, on ='userid', how = 'inner')
#compare
compare.to_csv(path +'newScripts/moodVector/moodVectorsData/PostsLyricsTranEmptyAllVar.csv')
corMatrix = compare.corr()
corMatrix.to_csv(path + 'newScripts/moodVector/moodVectorsData/PostsLyricsTrancorrelationMatrix(Empty).csv')
corMatrix

,emptyTrans,lyricsTran,lyricsEmpty,allTrans,ope,con,ext,agr,neu,swl,CESD_sum
emptyTrans,1.000000,-0.633202,-0.997930,NaN,0.038676,0.111529,-0.052871,0.145154,-0.176357,0.195584,-0.174408
lyricsTran,-0.633202,1.000000,0.582115,NaN,0.000151,0.104639,0.105847,-0.235110,-0.044332,0.038876,0.043670
lyricsEmpty,-0.997930,0.582115,1.000000,NaN,-0.040643,-0.125861,0.046748,-0.132955,0.188954,-0.208700,0.179594
allTrans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ope,0.038676,0.000151,-0.040643,NaN,1.000000,-0.125873,0.293143,0.108909,-0.250507,-0.072971,-0.242780
con,0.111529,0.104639,-0.125861,NaN,-0.125873,1.000000,0.068596,0.176327,-0.247231,0.379477,-0.320469
ext,-0.052871,0.105847,0.046748,NaN,0.293143,0.068596,1.000000,0.239230,-0.295430,0.149224,-0.303441
agr,0.145154,-0.235110,-0.132955,NaN,0.108909,0.176327,0.239230,1.000000,-0.370128,0.206110,-0.159412
neu,-0.176357,-0.044332,0.188954,NaN,-0.250507,-0.247231,-0.295430,-0.370128,1.000000,-0.406449,0.557362
swl,0.195584,0.038876,-0.208700,NaN,-0.072971,0.379477,0.149224,0.206110,-0.406449,1.000000,-0.404329


In [88]:
compareFreq = pd.merge(df, Var, on ='userid', how = 'left')
corMatrix = compareFreq.corr()
corMatrix.to_csv(path + 'newScripts/moodVector/moodVectorsData/PostsLyricsFreqMatrix(Empty).csv')
corMatrix

,emptyDays,lyrics,ope,con,ext,agr,neu,swl,CESD_sum
emptyDays,1.000000,-0.186476,-0.123095,0.196513,-0.089701,0.059745,0.036200,0.172899,-0.118434
lyrics,-0.186476,1.000000,-0.070261,-0.058389,0.043970,-0.150912,0.182004,-0.155665,0.155276
ope,-0.123095,-0.070261,1.000000,-0.125873,0.293143,0.108909,-0.250507,-0.072971,-0.242780
con,0.196513,-0.058389,-0.125873,1.000000,0.068596,0.176327,-0.247231,0.379477,-0.320469
ext,-0.089701,0.043970,0.293143,0.068596,1.000000,0.239230,-0.295430,0.149224,-0.303441
agr,0.059745,-0.150912,0.108909,0.176327,0.239230,1.000000,-0.370128,0.206110,-0.159412
neu,0.036200,0.182004,-0.250507,-0.247231,-0.295430,-0.370128,1.000000,-0.406449,0.557362
swl,0.172899,-0.155665,-0.072971,0.379477,0.149224,0.206110,-0.406449,1.000000,-0.404329
CESD_sum,-0.118434,0.155276,-0.242780,-0.320469,-0.303441,-0.159412,0.557362,-0.404329,1.000000


Now we write the valence vector as csv file 